# Carga del modelo

In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import numpy as np


C:\Users\Carla\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
model = gensim.models.Word2Vec.load("./doc2vec_wikipedia_es_pvdbow")
model.docvecs.vectors_docs.shape
model.most_similar(positive=['seguros'],topn=10)


C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('aseguradora', 0.6253476738929749),
 ('reaseguros', 0.5982040166854858),
 ('aseguradoras', 0.5916717052459717),
 ('rga', 0.5637242794036865),
 ('bancarios', 0.5485397577285767),
 ('asegurador', 0.5484205484390259),
 ('bancaria', 0.5317405462265015),
 ('crédit', 0.5307000875473022),
 ('reaseguradoras', 0.5265746116638184),
 ('banca', 0.5258691906929016)]

# Carga de Data

In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

bdtesis = client.BDTesis
preguntas = bdtesis.PreguntasSBS


In [4]:
#a minusculas
data = []
for x in bdtesis.PreguntasSBS.find():
    data.append(x['Pregunta'])
datamin =[]
for x in (data):
    datamin.append(x.lower()) 


In [5]:
#limpieza de puntuación
import re
import nltk
import string
cadpun = string.punctuation + '¿'
datapunc = []
for x in datamin:
    datapunc.append(x.translate(str.maketrans('', '', cadpun)))


In [6]:
#retirar stopwords
from nltk.corpus import stopwords
stops = stopwords.words('spanish')
dataclean = []
for x in datapunc:
    palabras = x.split()
    resultwords  = [palabra for palabra in palabras if palabra.lower() not in stops]
    dataclean.append(' '.join(resultwords))
    

In [7]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(dataclean)]

In [8]:
datamatrix = []
for x in dataclean:
    palabras = x.split()
    datamatrix.append(palabras)
datamatrix

[['sbs',
  'definirá',
  'codificaciones',
  'activos',
  'emitidos',
  'serie',
  'públicos'],
 ['cuántos', 'días', 'sbs', 'confirme', 'códigos', 'inscritos'],
 ['empresa',
  'podrá',
  'usar',
  'código',
  'propio',
  'interno',
  'recibir',
  'comunicación',
  'sbs'],
 ['sucede', 'tasa', 'renta', 'fija'],
 ['debe', 'colocarse', 'aplica', 'tasa', 'variable'],
 ['emisores',
  'vehículo',
  'emitir',
  'deuda',
  'domiciliada',
  'país',
  'ventajas',
  'tributarias',
  'considera',
  'país',
  'constituye',
  'vehículo',
  'país',
  'riesgo',
  'empresa'],
 ['emisores',
  'vehículo',
  'emitir',
  'deuda',
  'domiciliada',
  'país',
  'ventajas',
  'tributarias',
  'considera',
  'vehículo',
  'emisor',
  'empresa'],
 ['establecerán',
  'códigos',
  'entidades',
  'localesextranjeras',
  'menor',
  'fortaleza',
  'financiera',
  '“a”'],
 ['emisores',
  'reportan',
  'smv',
  'prima',
  'grupo',
  'económico',
  'figura',
  'página',
  'web',
  'entidad'],
 ['cuándo',
  'trate',
  'co

In [16]:
vectoresPF = []
for pregunta in datamatrix:
    vector = model.infer_vector(pregunta)
    vectoresPF.append(vector)
len(vectoresPF)

vectoresPF[1]

array([-0.05972479, -0.05568403, -0.06620022,  0.03403597, -0.10681933,
        0.09457041, -0.04286496,  0.27934974, -0.16100377,  0.1015491 ,
       -0.0491951 ,  0.01753034, -0.08895296, -0.01574496, -0.07045558,
       -0.03933155,  0.13583715, -0.14525233, -0.07920038,  0.05387066,
       -0.0800891 , -0.10262757,  0.09898335,  0.01738707, -0.13903375,
        0.06792646, -0.03814863,  0.01459619, -0.03002044, -0.06407254,
        0.10736439,  0.00437351, -0.00611262, -0.05426995,  0.02886648,
        0.14071448, -0.04003451,  0.03239725, -0.08547917, -0.03042855,
       -0.02808206,  0.08598024, -0.05827774,  0.10686806, -0.07153766,
        0.09019591,  0.1331394 , -0.04794298, -0.0370275 , -0.11865938,
       -0.09171094,  0.08000398,  0.04640245,  0.07607926,  0.11915452,
        0.04504719,  0.01540713, -0.08790739, -0.08153829,  0.11323284,
       -0.09634202, -0.03672686,  0.06270006,  0.03297488,  0.01199154,
       -0.01972903,  0.09560034, -0.08347893,  0.02087494,  0.12

In [15]:
#GUARDADO EN BD, SE CREA UNA COLUMNA NUEVA EN LA COLECCIÓN
cont = 0
for x in bdtesis.PreguntasSBS.find():
    bdtesis.PreguntasSBS.update({"_id": x['_id']}, {"$set": {"vector": vectoresPF[cont]}})
    cont = cont +1

C:\Users\Carla\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  after removing the cwd from sys.path.


InvalidDocument: Cannot encode object: array([ 4.31756936e-02, -6.54344708e-02, -4.59485427e-02,  4.55321148e-02,
       -5.73822297e-03, -1.68881994e-02, -1.43011481e-01,  4.63109836e-02,
       -7.83292800e-02,  1.74523443e-02,  2.77455207e-02,  1.83539465e-01,
        4.23698500e-02, -1.23132825e-01,  1.04603618e-01, -1.20356552e-01,
        1.18490599e-01,  2.83681378e-02, -1.86289728e-01, -3.72872241e-02,
       -1.02158070e-01, -1.83193758e-02, -1.36109730e-02, -4.04978618e-02,
       -9.78239626e-02, -1.94929987e-01, -1.33611962e-01,  6.11322746e-02,
       -6.09006956e-02, -9.06481668e-02, -8.72700661e-02,  1.47184074e-01,
        2.44940463e-02,  6.17906600e-02,  1.57844331e-02, -2.73675025e-02,
       -1.49782628e-01, -3.54204103e-02, -1.57477230e-01,  1.16944045e-01,
        8.10170025e-02,  5.77853955e-02, -4.34131101e-02,  7.82679990e-02,
       -5.86255454e-03,  2.95721833e-02, -1.19929463e-02, -1.07557222e-01,
        1.08033232e-01, -2.36659154e-01, -1.04918346e-01, -6.

In [16]:
vectoresPFnp = np.asarray(vectoresPF)
vectoresPFnp

array([[ 0.02352454, -0.14449023, -0.12243807, ...,  0.12982097,
        -0.00083952, -0.03285021],
       [-0.0755804 , -0.06061995, -0.14678694, ...,  0.11367293,
        -0.04748369,  0.00659025],
       [-0.05188518, -0.02256797, -0.0397667 , ...,  0.03472189,
         0.04389158, -0.18045145],
       ...,
       [ 0.0217593 , -0.04653136,  0.06336712, ...,  0.12126923,
        -0.04662034, -0.21836813],
       [ 0.09635577, -0.11174507, -0.04035002, ...,  0.24356122,
        -0.05343472, -0.05710325],
       [-0.05864929, -0.05560217, -0.04003491, ..., -0.01912575,
         0.07318867, -0.14376919]], dtype=float32)

In [15]:
modelPF = Doc2Vec(dm=0, vector_size=300, window=6, min_count=2, epochs=100, hs=1)
vocab = modelPF.build_vocab(tagged_data)

In [76]:
%time modelPF.train(tagged_data, total_examples=modelPF.corpus_count, epochs=modelPF.epochs)

Wall time: 1.44 s


In [81]:
print (modelPF.docvecs.most_similar(7))

[('70', 0.7830699682235718), ('8', 0.7035921812057495), ('150', 0.6834375858306885), ('98', 0.6160706281661987), ('1', 0.600631833076477), ('58', 0.5968820452690125), ('15', 0.5936374664306641), ('5', 0.5935928225517273), ('146', 0.5926030874252319), ('6', 0.5872544646263123)]


In [83]:
modelPF.docvecs.vectors_docs.shape

(151, 300)

# Carga como doc2vec

In [104]:
modeld2v= Doc2Vec.load("./doc2vec_wikipedia_es_pvdbow")


In [106]:
from gensim.models.doc2vec import Doc2Vec

test_data = word_tokenize("prueba de datos".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [-4.90295738e-02 -6.51554763e-03 -4.22625989e-02  4.95705232e-02
 -2.64223069e-02  4.56278352e-03  9.49633867e-03  4.32882048e-02
 -4.52837795e-02  3.97661142e-02  5.03136888e-02  3.36665940e-03
  3.26613151e-03  5.86011168e-03 -5.18063493e-02 -7.45031759e-02
  1.37174696e-01  2.49932669e-02 -5.25499098e-02 -5.81274852e-02
 -5.78674264e-02 -8.77256691e-03  2.36625057e-02  9.67941526e-03
 -2.14932896e-02 -1.62547722e-01  2.88621951e-02  1.16603216e-02
  2.28412859e-02  3.53063047e-02  5.85132949e-02  4.98251133e-02
 -7.89942294e-02  7.55206347e-02  2.78369617e-02  3.43146175e-02
  2.32476648e-02  4.53507900e-02 -1.64867137e-02 -2.42035612e-02
  1.33583978e-01  7.76500814e-03 -4.85584997e-02  3.95043828e-02
 -4.04941887e-02  6.67379200e-02  5.19856736e-02 -2.77740173e-02
 -6.03712350e-03 -7.44261220e-02  5.42298742e-02  8.98217708e-02
 -2.83276234e-02  3.17138247e-02  1.11978445e-02 -6.35849759e-02
 -9.27890763e-02 -3.52885239e-02  2.79444978e-02  6.34896132e-05
  1.87146792e-03